In [11]:
import pandas as pd # Make sure that you have openpyxl installed so that you can open newer excel files
import pandas.api.types as pdtypes
import numpy as np
import os 
from zipfile import is_zipfile, ZipFile
from datetime import datetime, date
from plotnine import ggplot, geom_line, aes, theme_minimal, geom_tile, geom_label, scale_fill_distiller, labs, theme
import plotly.graph_objects as go



In [12]:
def as_categorical(c, copy=True):
    if not pdtypes.is_categorical_dtype(c):
        c = pd.Categorical(c)
    elif copy:
        c = c.copy()
    return c

def cat_rev(c):
    c = as_categorical(c)
    c.reorder_categories(c.categories[::-1], ordered=True)
    return c

def cat_inorder(c, ordered=None):
    kwargs = {} if ordered is None else {'ordered': ordered}
    if isinstance(c, (pd.Series, pd.Categorical)):
        cats = c[~pd.isnull(c)].unique()
        if hasattr(cats, 'to_list'):
            cats = cats.to_list()
    elif hasattr(c, 'dtype'):
        cats = pd.unique(c[~pd.isnull(c)])
    else:
        cats = pd.unique([
            x for x, keep in zip(c, ~pd.isnull(c))
            if keep
        ])
    return pd.Categorical(c, categories=cats, **kwargs)

def extractFolder(inputFilesDir,unzippedFilesDir):
    for file in os.listdir(inputFilesDir):   # get the list of files
        if is_zipfile(inputFilesDir+file): # if it is a zipfile, extract it
            with ZipFile(inputFilesDir+file) as item: # treat the file as a zip
                item.extractall(unzippedFilesDir)  # extract it into a new folder

def cleanTermStructureSheet(filename,foldername,sheetName):
    df = pd.read_excel(foldername+filename, sheet_name=sheetName, header=1, engine='openpyxl')
    df.drop(['Unnamed: 0'], axis=1, inplace=True) # Blank column - artifact of the dataset
    df = df.iloc[8:,:] # Dropping all unnessary columns 
    df.rename(columns={"Main menu": "Tenor"}, inplace=True)
    df.set_index("Tenor", inplace=True)
    return df

def getDateFromFileName(filename):
    dateString = filename.split('_')[2]
    yearString = dateString[:4]
    monthString = dateString[4:6]
    dayString = dateString[6:]
    return date(int(yearString),int(monthString),int(dayString))

def getCountrieslist(inputFilesDir):
    return pd.read_csv(inputFilesDir+"countries.csv").columns.to_list()

def getFilteredFileList(unzippedFilesDir,fileType,sheetName):
    allFiles = os.listdir(unzippedFilesDir)
    filteredList = [file for file in allFiles if fileType in file]
    return filteredList 

def plotCurvesOverTime(coutriesDict, country, colourScale):
    # if you have time experiment with lines 
    # https://community.plotly.com/t/add-lines-to-3d-surface/36042/2
    countryData = pd.DataFrame(countriesDict[country]).sort_index(axis = 1)
    dates = countryData.columns
    tenors = countryData.index.values
    camera = dict(
        up = dict(x=0, y=0, z=1), 
        center = dict(x=0, y=0, z=0), 
        eye = dict(x=1.6, y=1.6, z=0.5 )
    )

    fig = go.Figure(data=[go.Surface(z=countryData.values, 
                                     x=dates, 
                                     y=tenors,
                                     opacity = 0.8,
                                     colorscale=colourScale, 
                                     colorbar_thickness = 25, 
                                     colorbar_len = 0.75, 
                                    
                      )])

    fig.update_layout(scene_camera = camera,
                    scene=dict(yaxis_title='Tenor',
                            xaxis_title='Date',
                            zaxis_title='Yield',
                            aspectratio = dict(x = 1, y = 1.4, z= 0.75)),
                    title=f'{country} Curve Progression since 2022', autosize=False,
                    width=1000, height=700,
                    margin=dict(l=50, r=20, b=10, t=50), 
                    paper_bgcolor="LightSteelBlue", 
                    )

    fig.show()

def correlationMatrixDataframeHelper(country,countriesDict,tenorsOfInterest):
    df = pd.DataFrame(countriesDict[country]).iloc[tenorsOfInterest,:].T
    newColumns = [f'{country}_{tenor}Y' for tenor in df.columns.values]
    df.columns = newColumns
    return df


def createCrossCountryCrossTenorCorrelation(countries, tenors, countriesDict):
    tenors = [tenor - 1 for tenor in tenors]
    elementsOfCorrelationMatrix = []
    for country in countries:
        elementsOfCorrelationMatrix.append(correlationMatrixDataframeHelper(country,countriesDict,tenors))
    correlationDataFrame = pd.concat(elementsOfCorrelationMatrix, axis = 1)
    crossTenorCountryCorrelation = np.round(correlationDataFrame.diff().dropna().corr(),2)
    return crossTenorCountryCorrelation


inputFilesDir = "inputData/"
unzippedFilesDir = "unzippedFiles/"
fileType = "Term_Structures"
sheetName = "RFR_spot_no_VA"

extractFolder(inputFilesDir,unzippedFilesDir)
countries = getCountrieslist(inputFilesDir)

countriesDict = {}
for country in countries:
    countriesDict[country] = {}

for filename in getFilteredFileList(unzippedFilesDir,fileType,sheetName):
    dateIndex = getDateFromFileName(filename)
    allTermStructures = cleanTermStructureSheet(filename,unzippedFilesDir,sheetName)
    for country in countries:
        countriesDict[country][dateIndex] =  allTermStructures.loc[:,country]


In [13]:
plotCurvesOverTime(countriesDict,"United States", "Reds")

In [14]:
plotCurvesOverTime(countriesDict,"Euro", "Blues")

In [15]:
countriesForCorrelation = ["United States", "Euro", "United Kingdom", "China"]
tenorsOfInterest = [2, 5, 10, 20, 30]
tenorsOfInterest = [tenor - 1 for tenor in tenorsOfInterest]

elementsOfCorrelationMatrix = []
for country in countriesForCorrelation:
    elementsOfCorrelationMatrix.append(correlationMatrixDataframeHelper(country,countriesDict,tenorsOfInterest))
correlationDataFrame = pd.concat(elementsOfCorrelationMatrix, axis = 1)
crossTenorCountryCorrelation = np.round(correlationDataFrame.diff().dropna().corr(),2)


In [16]:
def createCrossCountryCrossTenorCorrelation(countries, tenors, countriesDict):
    tenors = [tenor - 1 for tenor in tenors]
    elementsOfCorrelationMatrix = []
    for country in countries:
        elementsOfCorrelationMatrix.append(correlationMatrixDataframeHelper(country,countriesDict,tenors))
    correlationDataFrame = pd.concat(elementsOfCorrelationMatrix, axis = 1)
    crossTenorCountryCorrelation = correlationDataFrame.diff().dropna().corr()
    return crossTenorCountryCorrelation


In [17]:
countriesForCorrelation = ["United States", "Euro", "United Kingdom", "China"]
tenorsOfInterest = [2, 5, 10, 20, 30]
corrMatrix =  createCrossCountryCrossTenorCorrelation(countriesForCorrelation, tenorsOfInterest, countriesDict)
corrMatrix 

,United States_2Y,United States_5Y,United States_10Y,United States_20Y,United States_30Y,Euro_2Y,Euro_5Y,Euro_10Y,Euro_20Y,Euro_30Y,United Kingdom_2Y,United Kingdom_5Y,United Kingdom_10Y,United Kingdom_20Y,United Kingdom_30Y,China_2Y,China_5Y,China_10Y,China_20Y,China_30Y
United States_2Y,1.000000,0.960711,0.889943,0.827855,0.783755,0.867976,0.848161,0.841468,0.790093,0.760022,0.609127,0.700191,0.782437,0.736380,0.700313,-0.269068,-0.196846,-0.237661,-0.252996,-0.237127
United States_5Y,0.960711,1.000000,0.980255,0.947802,0.919665,0.824106,0.781800,0.837472,0.886623,0.875593,0.640432,0.724117,0.853139,0.854510,0.830644,-0.391341,-0.337563,-0.383097,-0.398282,-0.377854
United States_10Y,0.889943,0.980255,1.000000,0.991608,0.977255,0.747178,0.694249,0.800988,0.925027,0.929340,0.614285,0.689999,0.856797,0.899618,0.887300,-0.434622,-0.394421,-0.443609,-0.458140,-0.433918
United States_20Y,0.827855,0.947802,0.991608,1.000000,0.996202,0.681528,0.611165,0.745504,0.922519,0.938824,0.598835,0.663426,0.850444,0.919799,0.915747,-0.471972,-0.442985,-0.493209,-0.507062,-0.481480
United States_30Y,0.783755,0.919665,0.977255,0.996202,1.000000,0.640103,0.551726,0.699812,0.911874,0.936661,0.596094,0.651332,0.848527,0.933700,0.935030,-0.498391,-0.478991,-0.528969,-0.542019,-0.515889
Euro_2Y,0.867976,0.824106,0.747178,0.681528,0.640103,1.000000,0.923481,0.853133,0.749159,0.709314,0.669997,0.781186,0.785802,0.674086,0.621539,-0.139680,-0.099833,-0.114130,-0.115047,-0.098870
Euro_5Y,0.848161,0.781800,0.694249,0.611165,0.551726,0.923481,1.000000,0.952350,0.749865,0.678749,0.407986,0.558086,0.584597,0.485654,0.434368,0.109773,0.166813,0.145344,0.136130,0.147713
Euro_10Y,0.841468,0.837472,0.800988,0.745504,0.699812,0.853133,0.952350,1.000000,0.894192,0.838719,0.350515,0.500561,0.609152,0.582595,0.547742,0.093405,0.145878,0.108232,0.095586,0.115432
Euro_20Y,0.790093,0.886623,0.925027,0.922519,0.911874,0.749159,0.749865,0.894192,1.000000,0.992910,0.452180,0.557723,0.749352,0.813348,0.803313,-0.156785,-0.133709,-0.183776,-0.195723,-0.166366
Euro_30Y,0.760022,0.875593,0.929340,0.938824,0.936661,0.709314,0.678749,0.838719,0.992910,1.000000,0.483457,0.570888,0.770372,0.848589,0.843080,-0.221554,-0.202614,-0.254428,-0.264815,-0.232292


In [18]:
countriesForCorrelation = ["United States", "Euro"]
tenorsOfInterest = [2, 5]
corrMatrix =  createCrossCountryCrossTenorCorrelation(countriesForCorrelation, tenorsOfInterest, countriesDict)
corrMatrix 

,United States_2Y,United States_5Y,Euro_2Y,Euro_5Y
United States_2Y,1.000000,0.960711,0.867976,0.848161
United States_5Y,0.960711,1.000000,0.824106,0.781800
Euro_2Y,0.867976,0.824106,1.000000,0.923481
Euro_5Y,0.848161,0.781800,0.923481,1.000000


In [19]:
# # from itertools import chain, product


# import pandas.api.types as pdtypes
# # from pandas.core.algorithms import value_counts

# def as_categorical(c, copy=True):
#     if not pdtypes.is_categorical_dtype(c):
#         c = pd.Categorical(c)
#     elif copy:
#         c = c.copy()
#     return c

# def cat_rev(c):
#     c = as_categorical(c)
#     c.reorder_categories(c.categories[::-1], ordered=True)
#     return c

# def cat_inorder(c, ordered=None):
#     kwargs = {} if ordered is None else {'ordered': ordered}
#     if isinstance(c, (pd.Series, pd.Categorical)):
#         cats = c[~pd.isnull(c)].unique()
#         if hasattr(cats, 'to_list'):
#             cats = cats.to_list()
#     elif hasattr(c, 'dtype'):
#         cats = pd.unique(c[~pd.isnull(c)])
#     else:
#         cats = pd.unique([
#             x for x, keep in zip(c, ~pd.isnull(c))
#             if keep
#         ])
#     return pd.Categorical(c, categories=cats, **kwargs)


In [20]:

(
    corrMatrix
    .melt(ignore_index = False)
    .reset_index()
    .set_axis(labels= ['var1', 'var2', 'value'], axis = 1)
    .assign(lab_text = lambda x: np.round(x['value'],2))
    .assign(
        var1 = lambda x : cat_inorder(x['var1']),
        var2 = lambda x : cat_rev(cat_inorder(x['var2']))
    )  
)

,var1,var2,value,lab_text
0,United States_2Y,United States_2Y,1.000000,1.00
1,United States_5Y,United States_2Y,0.960711,0.96
2,Euro_2Y,United States_2Y,0.867976,0.87
3,Euro_5Y,United States_2Y,0.848161,0.85
4,United States_2Y,United States_5Y,0.960711,0.96
5,United States_5Y,United States_5Y,1.000000,1.00
6,Euro_2Y,United States_5Y,0.824106,0.82
7,Euro_5Y,United States_5Y,0.781800,0.78
8,United States_2Y,Euro_2Y,0.867976,0.87
9,United States_5Y,Euro_2Y,0.824106,0.82
